In [ ]:
# Copyright (c) 2025, ETH Zurich

# This analysis notebooks require data that is generated from other notebooks

## Namely:

- DRIE_filtered.ipynb
- DRIE_unfiltered.ipynb
- DRIE_XLIGA_filtered.ipynb
- DRIE_XLIGA_unfiltered.ipynb
- XLIGA_filtered.ipynb
- XLIGA_unfiltered.ipynb
- Missing_Trenches_Simulation.ipynb
- Simulation_Crack.ipynb

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.stats as stats
import scipy.ndimage
from tqdm import tqdm
import spekpy as spk
import h5py
import json
from scipy import interpolate

from scipy.interpolate import griddata

In [ ]:
rave_sim_dir = Path('../rave-sim').resolve()
simulations_dir = Path('/PATH/TO/DATA/WHERE/SIMULATIONS/ARE/STORED/')
#simulations_dir = Path('../data/Measurement_Simulations/')
scratch_dir = simulations_dir


sys.path.insert(0, str(rave_sim_dir / "nist_lookup"))
from nist_lookup.xraydb_plugin import xray_delta_beta

In [ ]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import config
import util
import propagation


In [ ]:
# matplotlib style
plt.style.use("default")

# set FIGWIDTH to latex's \textwidth
FIGWIDTH = 3
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 5
plt.rcParams["figure.figsize"] = (FIGWIDTH, FIGWIDTH * 2 / 3)
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.constrained_layout.use"] = "True"

# images
plt.rcParams["image.interpolation"] = "bicubic"
plt.rcParams["image.cmap"] = "Greys_r"

# axes
# plt.rcParams["axes.spines.right"] = False
# plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.edgecolor"] = "0.7"
plt.rcParams["axes.linewidth"] = "1"

# legend
plt.rcParams["legend.frameon"] = False

plt.rcParams["lines.markersize"] = 3
# plt.rcParams["lines.markerfacecolor"] = "white"
# Okabe-Ito palette
plt.rcParams["axes.prop_cycle"] = plt.cycler(
    color=[
        "#000000",
        "#E69F00",
        "#56B4E9",
        "#009E73",
        "#F0E442",
        "#0072B2",
        "#D55E00",
        "#CC79A7",
    ],
    #marker=["o", "^", "s", "p", "D", "v", "v", "d"],
)

plt.rcParams['axes.grid'] = False

In [ ]:
h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
c_0 = 299792458 # speed of light in m / s
eV_to_joule = 1.602176634*10**(-19)
N_A = 6.02214086 * 10**23 #[1/mol]

# Disclaimer detector response 
For the following code lines a detector response is required. The plots in the paper: Quantification of the effects of grating defects on the sensitivity of a Talbot-Lau interferometer
The files for the detector response are not included but can be generated using DOI: <a href="https://aapm.onlinelibrary.wiley.com/doi/10.1002/mp.12863">10.1002/mp.12863</a>

For the detector response: function_name – interpolate_response_function the following parameters could be used:
- $r_0 = 10$
- $\sigma_e = 0.5\,keV$
- $\Delta_{pix} = 100 \,\mu m$
- $dz = 750\,\mu m$
- $\rho_{PCD} = 5.85\,g/cm^2$
- $NI = 1$
- $E_{th1} = 11 keV$


In [ ]:
def calculate_G1_height(eng):
    # constants
    h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
    c_0 = 299792458 # speed of light in m / s
    eV_to_joule = 1.602176634*10**(-19)
    N_A = 6.02214086 * 10**23 #[1/mol]
    
    lambda_ = h * c_0 / (eng*eV_to_joule)
    delta_diff = xray_delta_beta('Si', 2.34, eng)[0]
    height = np.pi  * lambda_ / (2*np.pi * delta_diff)

    return height


def signal_retrieval_least_squares(data, period=None, axis=-1):
    if axis != -1:
        data = np.moveaxis(data, axis, -1)

    nsteps = data.shape[-1]

    if period is None:
        period = nsteps

    phi = np.linspace(0, 2 * np.pi * nsteps / (period), nsteps, endpoint=False)
    M = np.c_[np.sin(phi), np.cos(phi), np.ones(nsteps)]
    res, chi2, _rank, _sing_vals = np.linalg.lstsq(M, data.reshape((-1, nsteps)).T, rcond=-1)

    res = res.T.reshape((*data.shape[:-1], -1))

    dabs = res[...,2]
    dphase = -np.arctan2(res[...,0], res[...,1])
    dvis = np.sqrt(res[...,0]**2 + res[...,1]**2) / dabs

    # normalization to the total number of counts
    dabs *= nsteps

    return dabs, dphase, dvis, np.nanmean(chi2)


def calculate_pixel_intensity(x, fringe, pxEdges, statistics = 'sum'):
    fringeStats = stats.binned_statistic(x, fringe, bins=pxEdges, statistic = statistics)
    return fringeStats.statistic

def perform_binned_signal_retrieval(x, wf, pxSize, nrSteps, plot_curve = True):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = []
    for i in range(nrSteps):
        int_px.append(calculate_pixel_intensity(x, wf[i,:], pxEdges))
    int_px = np.asarray(int_px)

    trans, phase, vis, _ = signal_retrieval_least_squares(int_px, period = nrSteps, axis = 0)
    if plot_curve:
        plot_curves(trans, phase, vis, pxEdges)
    return int_px, trans, phase, vis, pxEdges

def plot_curves(trans, phase, vis, pxEdges):

    fig, axs = plt.subplots(figsize=(15,6), sharex = True, nrows = 1, ncols = 2)
    axs[0].plot(pxEdges[1:], trans, label = 'Transmission')
    axs[0].set_title('Transmission')
    axs[1].plot(pxEdges[1:], vis, label = 'Visibility')
    axs[1].set_title('Visibility')

def perform_binning(x, wf, pxSize):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = calculate_pixel_intensity(x, wf, pxEdges, statistics = 'mean')
    return int_px


    #np.random.seed(1)
    
    sample_energy = np.random.choice(x, p=y)
    if sample_energy >= low_threshold:
        return True
    else:
        return False
    
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def calculate_SNR(wavefronts, detector_x, phase_steps):
    summed_wf = np.zeros_like(wavefronts[0][0])
        
    for point in wavefronts:
        wf, x_point, eng = point
        
        summed_wf += wf

    return summed_wf

def get_subdict(dict_, idx):
    sub_dict = {}
    for key in dict_.keys():
        sub_dict[key] = dict_[key][idx]
    return sub_dict

def calc_Vis_theoretical(eng, Edes,m):
    V = 2/np.pi * np.abs(np.sin(np.pi / 2 * Edes / eng)**2 * np.sin(m * np.pi / 2 * Edes / eng))
    return V

################
### IN THE PAPER THE OFFICIAL DETECTOR RESPONSE FROM DECTRIS HAS BEEN UTILIZED. DUE TO IP WE DO NOT SHARE THE DETECTOR RESPONSE.
### A DETECTOR RESPONSE CAN BE GENERATED WITH: PcTK
###
################


def interpolate_response_function(energy, low_threshold):
    srf_mat = scipy.io.loadmat('SRF_model_12_1TH.mat')['SRF_all']
    energies_mat = np.arange(0, srf_mat.shape[0])*1e3 
    #low_threshold = int(low_threshold / 1e3)
    #upper_threshold = int(upper_threshold / 1e3)
    energy = int(energy / 1e3)
    y = srf_mat[:, energy]
    x = energies_mat[:]
    y = y / np.sum(y)

    idx = find_nearest(x, low_threshold)

    return np.sum(y[idx:])
    

def mu_h2o(eng):
    lambda_ = h * c_0 / (eng*eV_to_joule)
    beta = xray_delta_beta('H2O', 1.0, eng)[1]
    return 4 * np.pi / lambda_ * beta


# Disclaimer detector response 
For the following code lines a detector response is required to generate the same plots as in: Quantifying grating defects in X-ray Talbot-Lau interferometer through a comparative study of two fabrication techniques

The files for the detector response are not included but can be generated using DOI: <a href="https://aapm.onlinelibrary.wiley.com/doi/10.1002/mp.12863">10.1002/mp.12863</a>

The package can be requested on the following link: <a href="https://pctk.jhu.edu/">https://pctk.jhu.edu/</a>

Example parameters for a detector response – SRF_model_12_1TH the following parameters were used:
- $r_0 = 10$
- $\sigma_e = 0.5\,keV$
- $\Delta_{pix} = 100 \,\mu m$
- $dz = 750\,\mu m$
- $\rho_{PCD} = 5.85\,g/cm^2$
- $NI = 1$
- $E_{th1} = 11 keV$


In [ ]:
################
### IN THE PAPER THE OFFICIAL DETECTOR RESPONSE FROM DECTRIS HAS BEEN UTILIZED. DUE TO IP WE DO NOT SHARE THE DETECTOR RESPONSE.
### A DETECTOR RESPONSE CAN BE GENERATED WITH: PcTK
###############


def interpolate_response_function(energy, low_threshold):
    srf_mat = scipy.io.loadmat('<PATH_TO_DETECTOR_RESPONSE>/SRF_model_12_1TH.mat')['SRF_all']
    energies_mat = np.arange(0, srf_mat.shape[0])*1e3 
    #low_threshold = int(low_threshold / 1e3)
    #upper_threshold = int(upper_threshold / 1e3)
    energy = int(energy / 1e3)
    y = srf_mat[:, energy]
    x = energies_mat[:]
    y = y / np.sum(y)

    idx = find_nearest(x, low_threshold)

    return np.sum(y[idx:])
    

# DRIE - Etched Gratings

## Unfiltered spectrum

Data generated by: DRIE_unfiltered.ipynb is required

In [ ]:
drie_unfiltered_path = Path('PATH/TO//DRIE_unfiltered_path/')

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_unfiltered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_unfiltered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_drie_uf, phase_ref, vis_ref_drie_uf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_drie_uf, phase_orig, vis_orig_drie_uf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




## 3mm Al filtered spectrum

Data generated by: DRIE_filtered.ipynb is required

In [ ]:
drie_filtered_path = Path('PATH/TO/DRIE_filtered_path/')

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_filtered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_filtered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_drie_f, phase_ref, vis_ref_drie_f, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
#dvis_ref_drie_f = (np.max(int_px, axis = 0) - np.min(int_px, axis = 0)) / (np.max(int_px, axis = 0) + np.min(int_px, axis = 0)) 
int_px, trans_orig_drie_f, phase_orig, vis_orig_drie_f, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)
#dvis_orig_drie_f = (np.max(int_px, axis = 0) - np.min(int_px, axis = 0)) / (np.max(int_px, axis = 0) + np.min(int_px, axis = 0)) 




## With 3mm Al and 10cm water

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_filtered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_filtered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * np.exp(-mu_h2o(wave[2])*0.1) * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_drie_wf, phase_ref, vis_ref_drie_wf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_drie_wf, phase_orig, vis_orig_drie_wf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




## Visibility spectrum DRIE

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_unfiltered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_unfiltered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

phase_steps = 5

vis_drie_all = []
energies_drie = []
for wave in tqdm(wavefronts_ref):
    energies_drie.append(wave[2])
    convolved_ref = []

    int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, wave[0][:, :], pxSize, phase_steps, plot_curve = False)


    vis_ref[vis_ref > 1] = 1
    vis_drie_all.append(np.min(vis_ref))
    vis_ref[vis_ref > 1] = 1
vis_drie_all = np.asarray(vis_drie_all)
energies_drie = np.asarray(energies_drie)

sorted_idxs_eng = np.argsort(energies_drie)
energies_drie = energies_drie[sorted_idxs_eng]
vis_drie_all = vis_drie_all[sorted_idxs_eng]


In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_filtered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_filtered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

phase_steps = 5

vis_drie_all_f = []
energies_drie_f = []
for wave in tqdm(wavefronts_ref):
    energies_drie_f.append(wave[2])
    convolved_ref = []

    int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, wave[0][:, :], pxSize, phase_steps, plot_curve = False)


    vis_ref[vis_ref > 1] = 1
    vis_drie_all_f.append(np.min(vis_ref))
    vis_ref[vis_ref > 1] = 1
vis_drie_all_f = np.asarray(vis_drie_all_f)
energies_drie_f = np.asarray(energies_drie_f)

sorted_idxs_eng = np.argsort(energies_drie_f)
energies_drie_f = energies_drie_f[sorted_idxs_eng]
vis_drie_all_f = vis_drie_all_f[sorted_idxs_eng]


In [ ]:
plt.plot(energies_drie, calc_Vis_theoretical(energies_drie, 46000, 3))
plt.plot(energies_drie, vis_drie_all, label ='new')
plt.plot()
#plt.axvline(45100)
#plt.axhline(0.36)

# X-LIGA Gratings with 180um height

## Unfiltered spectrum

Data generated by: XLIGA_unfiltered.ipynb is required

In [ ]:
liga_unfiltered_path = Path('PATH/TO/X-LIGA_unfiltered_PATH/')

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_unfiltered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_unfiltered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_liga_uf, phase_ref, vis_ref_liga_uf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_liga_uf, phase_orig, vis_orig_liga_uf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_unfiltered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_unfiltered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

phase_steps = 5

vis_liga_all = []
energies_liga = []
for wave in tqdm(wavefronts_ref):
    energies_liga.append(wave[2])
    convolved_ref = []

    int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, wave[0][:, :], pxSize, phase_steps, plot_curve = False)

    vis_ref[vis_ref > 1] = 1
    vis_liga_all.append(np.min(vis_ref))
    #vis_ref[vis_ref > 1] = 1
vis_liga_all = np.asarray(vis_liga_all)
energies_liga = np.asarray(energies_liga)

sorted_idxs_eng = np.argsort(energies_liga)
energies_liga = energies_liga[sorted_idxs_eng]
vis_liga_all = vis_liga_all[sorted_idxs_eng]


## 3mm Al filtered spectrum

Data generated by: XLIGA_filtered.ipynb is required

In [ ]:
liga_filtered_path = Path('PATH/TO/XLIGA_filtered_PATH/)

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_filtered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_filtered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

phase_steps = 5

vis_liga_all_f = []
energies_liga_f = []
for wave in tqdm(wavefronts_ref):
    energies_liga_f.append(wave[2])
    convolved_ref = []

    int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, wave[0][:, :], pxSize, phase_steps, plot_curve = False)


    vis_ref[vis_ref > 1] = 1
    vis_liga_all_f.append(np.min(vis_ref))
    #vis_ref[vis_ref > 1] = 1
vis_liga_all_f = np.asarray(vis_liga_all_f)
energies_liga_f = np.asarray(energies_liga_f)

sorted_idxs_eng = np.argsort(energies_liga_f)
energies_liga_f = energies_liga_f[sorted_idxs_eng]
vis_liga_all_f = vis_liga_all_f[sorted_idxs_eng]


In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_filtered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_filtered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_liga_f, phase_ref, vis_ref_liga_f, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_liga_f, phase_orig, vis_orig_liga_f, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




## With 3mm Al and 10cm water


In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_filtered_path) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_filtered_path, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000) * np.exp(-mu_h2o(wave[2])*0.1)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_liga_wf, phase_ref, vis_ref_liga_wf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_liga_wf, phase_orig, vis_orig_liga_wf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




## Plot visibility spectrum DRIE vs. LIGA

In [ ]:
color=[
    "#000000",
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#F0E442",
    "#0072B2",
    "#D55E00",
    "#CC79A7",
],

In [ ]:

# There are numerical instabilities with that approach for the visibility spectrum
#vis_drie_all[842:845] = 0.298
#vis_drie_all[847] = 0.269

plt.rcParams["font.family"] = "serif"

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(energies_liga, vis_liga_all, c = "#000000", linewidth = 3, label="X-LIGA")
ax.plot(energies_liga_f[-10:], vis_liga_all_f[-10:], c = "#000000", linewidth = 3)

ax.plot(energies_drie, vis_drie_all, c = "#E69F00", linewidth = 3, label="DRIE")
ax.plot(energies_drie_f[-10:], vis_drie_all_f[-10:], c = "#E69F00", linewidth = 3)
ax.xaxis.set_tick_params(labelsize=15)
ax.yaxis.set_tick_params(labelsize=15)

#ax.axvline(energies_drie[830])
plt.xlabel('Energy [eV]', fontsize = 15)
plt.ylabel('Visibility', fontsize = 15)
plt.legend(fontsize = 12)
ax.set_title('Visibility Spectrum', fontsize = 18)
plt.grid('on')
plt.savefig('Visibility_spectrum.png', bbox_inches = 'tight')
plt.rcParams["font.family"] = "serif"

# DRIE and X-LIGA 180um height

## Unfiltered spectrum

Data generated by: DRIE_XLIGA_unfiltered.ipynb is required

In [ ]:
drie_liga_unfiltered = Path('/PATH/TO/DRIE_XLIGA_unfiltered/)

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_liga_unfiltered) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_liga_unfiltered, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_drie_liga_uf, phase_ref, vis_ref_drie_liga_uf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_drie_liga_uf, phase_orig, vis_orig_drie_liga_uf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




## 3mm Al filtered spectrum

Data generated by: DRIE_XLIGA_filtered.ipynb is required

In [ ]:
drie_liga_filtered = Path('/PATH/TO/DRIE_XLIGA_filtered/')

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_liga_filtered) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_liga_filtered, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_drie_liga_f, phase_ref, vis_ref_drie_liga_f, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_drie_liga_f, phase_orig, vis_orig_drie_liga_f, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




## With 3mm Al and 10cm water

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(drie_liga_filtered) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(drie_liga_filtered, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000) * np.exp(-mu_h2o(wave[2])*0.1)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_drie_liga_wf, phase_ref, vis_ref_drie_liga_wf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_drie_liga_wf, phase_orig, vis_orig_drie_liga_wf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




# X-LIGA and DRIE 

## Unfiltered spectrum

Data generated by: DRIE_XLIGA_unfiltered.ipynb is required

In [ ]:
liga_drie_unfiltered = Path('/PATH/TO/XLIGA_DRIE_unfiltered')

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_drie_unfiltered) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_drie_unfiltered, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_liga_drie_uf, phase_ref, vis_ref_liga_drie_uf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_liga_drie_uf, phase_orig, vis_orig_liga_drie_uf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




In [ ]:
plt.plot(trans_ref_liga_drie_uf, label = 'LIGA-DRIE')
plt.plot(trans_ref_drie_uf, label = 'DRIE')

plt.plot(trans_ref_drie_liga_uf, label ='DRIE-LIGA')
plt.plot(trans_ref_liga_uf, label =  'LIGA')
plt.legend()



## 3mm Al filtered spectrum

Data generated by: DRIE_XLIGA_filtered.ipynb is required

In [ ]:
liga_drie_filtered = Path('/PATH/TO/XLIGA_DRIE_filtered')

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_drie_filtered) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_drie_filtered, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_liga_drie_f, phase_ref, vis_ref_liga_drie_f, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_liga_drie_f, phase_orig, vis_orig_liga_drie_f, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




## With 3mm Al and 10cm water

In [ ]:

pxSize = 75e-6
config_dict = config.load(Path(str(liga_drie_filtered) + '/config.yaml'))
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
pixel_rectangle = np.abs(detector_x) <= pxSize

wavefronts_ref = util.load_wavefronts_filtered(liga_drie_filtered, x_range=None, energy_range=[11000, 70000])
wf_ref = np.zeros_like(wavefronts_ref[0][0])

energies = []
for wave in wavefronts_ref:
    energies.append(wave[2])
    wf_ref += wave[0] * interpolate_response_function(wave[2], 11000) * np.exp(-mu_h2o(wave[2]) * 0.1)

phase_steps = 5
del wavefronts_ref

convolved_ref = []
for i in range(phase_steps):
    convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))

convolved_ref = np.asarray(convolved_ref)

int_px, trans_ref_liga_drie_wf, phase_ref, vis_ref_liga_drie_wf, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
int_px, trans_orig_liga_drie_wf, phase_orig, vis_orig_liga_drie_wf, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)




In [ ]:
liga_mean_uf = np.min(vis_orig_liga_uf[18:33])
drie_mean_uf = np.min(vis_orig_drie_uf[18:33])
drie_liga_mean_uf = np.min(vis_orig_drie_liga_uf[18:33])
liga_drie_mean_uf = np.min(vis_ref_liga_drie_uf[18:33])


liga_mean_f = np.min(vis_orig_liga_f[18:33])
drie_mean_f = np.min(vis_orig_drie_f[18:33])
drie_liga_mean_f = np.min(vis_orig_drie_liga_f[18:33])
liga_drie_mean_f = np.min(vis_ref_liga_drie_f[18:33])

liga_mean_wf = np.min(vis_orig_liga_wf[18:33])
drie_mean_wf = np.min(vis_orig_drie_wf[18:33])
drie_liga_mean_wf = np.min(vis_orig_drie_liga_wf[18:33])
liga_drie_mean_wf = np.min(vis_ref_liga_drie_wf[18:33])

# Compare to values from measurement

In [ ]:
data_f = []
with h5py.File('../data/Flats/Visibility_Measurement_Filtered_Gauss.h5', 'r') as h5:
    data_f.append(h5['LIGA'][:])
    data_f.append(h5['DRIE-LIGA'][:])
    data_f.append(h5['LIGA-DRIE'][:])
    data_f.append(h5['DRIE'][:])

data_wf = []
with h5py.File('../data/Flats/Visibility_Measurement_Unfiltered_Gauss.h5', 'r') as h5:
    data_wf.append(h5['LIGA'][:])
    data_wf.append(h5['DRIE-LIGA'][:])
    data_wf.append(h5['LIGA-DRIE'][:])
    data_wf.append(h5['DRIE'][:])
    
data_f_w = []
with h5py.File('../data/Flats/Visibility_Measurement_water_Gauss.h5', 'r') as h5:
    data_f_w.append(h5['LIGA'][:])
    data_f_w.append(h5['DRIE-LIGA'][:])
    data_f_w.append(h5['LIGA-DRIE'][:])
    data_f_w.append(h5['DRIE'][:])

In [ ]:

#Imports
import matplotlib.patches as mpatches
import matplotlib.lines as lines

In [ ]:
#data_boxplot = data_f[0,:]

plt.figure(figsize=(4,3))


#plt.sca(axs[0])

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    #plt.setp(bp['whiskers'], color=color)
    #plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color='black')

ticks = ['X-LIGA', 'DRIE/X-LIGA', 'X-LIGA/DRIE', 'DRIE']

bpl = plt.boxplot(data_f, positions=np.array(range(len(data_f_w)))*0.7, sym='', widths=0.15, whis = [1,99], patch_artist = True)
bpm = plt.boxplot(data_wf, positions=np.array(range(len(data_wf)))*0.7-0.2, sym='', widths=0.15, whis = [1,99], patch_artist = True)
bpr = plt.boxplot(data_f_w, positions=np.array(range(len(data_f_w)))*0.7+0.2, sym='', widths=0.15, whis = [1,99], patch_artist = True)

colors = ["#E69F00", "#56B4E9", "#009E73"]

set_box_color(bpl, colors[0]) # colors are from http://colorbrewer2.org/
set_box_color(bpm, colors[1])
set_box_color(bpr, colors[2])

# draw temporary red and blue lines and use them to create a legend
plt.plot([], c=colors[1], label='70kVp W spectrum')
plt.plot([], c=colors[0], label='With 3mm Al filter')
plt.plot([], c=colors[2], label='With filter and 10cm water')

plt.xticks(np.array(range(len(data_wf)))*0.7, ticks)
plt.xlim(-0.35, 2.45)
#plt.tight_layout()

plt.ylabel('Visibility', fontsize = 10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)


# Numbers behind indicate bridge fraction

plt.scatter(0-0.2, liga_mean_uf*0.91, marker = 'o', c = colors[1], s= 50, edgecolors='black')
plt.scatter(0.7-0.2, drie_liga_mean_uf*0.91, marker = 'o', c = colors[1], s=50, edgecolors='black')
plt.scatter(2.1-0.2, drie_mean_uf*0.99, marker = 'o', c= colors[1], s = 50, edgecolors='black')
plt.scatter(1.4-0.2, liga_drie_mean_uf*0.99, marker = 'o', c= colors[1], s = 50, edgecolors='black')


plt.scatter(0, liga_mean_f*0.91, marker = 'o', c = colors[0], s= 50, edgecolors='black')
plt.scatter(0.7, drie_liga_mean_f*0.91, marker = 'o', c = colors[0], s=50, edgecolors='black')
plt.scatter(2.1, drie_mean_f*0.99, marker = 'o', c= colors[0], s = 50, edgecolors='black')
plt.scatter(1.4, liga_drie_mean_f*0.99, marker = 'o', c= colors[0], s = 50, edgecolors='black')


plt.scatter(0+0.2, liga_mean_wf*0.91, marker = 'o', c = colors[2], s= 50, edgecolors='black')
plt.scatter(0.7+0.2, drie_liga_mean_wf*0.91, marker = 'o', c = colors[2], s=50, edgecolors='black')
plt.scatter(2.1+0.2, drie_mean_wf*0.99, marker = 'o', c= colors[2], s = 50, edgecolors='black')
plt.scatter(1.4+0.2, liga_drie_mean_wf*0.99, marker = 'o', c= colors[2], s = 50, edgecolors='black')


plt.axvline(0.35,  color='gray', linewidth=0.5)
plt.axvline(0.35 + 0.7,  color='gray', linewidth=0.5)
plt.axvline(0.35 + 2*0.7,  color='gray', linewidth=0.5)

ax = plt.gca()
legend_elements =  lines.Line2D([], [], marker='o', color='w', label='Simulations',  markerfacecolor='black', markersize=8)
# where some data has already been plotted to ax
handles, labels = ax.get_legend_handles_labels()
handles.append(legend_elements) 
# plot the legend
plt.legend(handles=handles, loc='upper left', fontsize = 6)


ymin, ymax = plt.gca().get_ylim()

#plt.legend(fontsize=6)

#plt.grid('off')
#plt.grid(axis='y')

plt.title('Visibility distribution comparison', fontsize = 10)
#plt.savefig('Visibility_Boxplots.png', bbox_inches = 'tight')

# Sensitivity

Due to IP protection the sensitivity can not be loaded to the public. Access can be granted upon reasonable request

In [ ]:
# data_sf = []
# with h5py.File('../data/Flats/Sensitivity_Measurement_Filtered_Gauss.h5', 'r') as h5:
#     data_sf.append(h5['LIGA'][:])
#     data_sf.append(h5['DRIE-LIGA'][:])
#     data_sf.append(h5['LIGA-DRIE'][:])
#     data_sf.append(h5['DRIE'][:])

# data_swf = []
# with h5py.File('../data/Flats/Sensitivity_Measurement_Unfiltered_Gauss.h5', 'r') as h5:
#     data_swf.append(h5['LIGA'][:])
#     data_swf.append(h5['DRIE-LIGA'][:])
#     data_swf.append(h5['LIGA-DRIE'][:])
#     data_swf.append(h5['DRIE'][:])
    
# data_sf_w = []
# with h5py.File('../data/Flats/Sensitivity_Measurement_water_Gauss.h5', 'r') as h5:
#     data_sf_w.append(h5['LIGA'][:])
#     data_sf_w.append(h5['DRIE-LIGA'][:])
#     data_sf_w.append(h5['LIGA-DRIE'][:])
#     data_sf_w.append(h5['DRIE'][:])

In [ ]:
# #data_boxplot = data_f[0,:]

# plt.figure(figsize=(4,3))


# #plt.sca(axs[0])

# def set_box_color(bp, color):
#     plt.setp(bp['boxes'], color=color)
#     #plt.setp(bp['whiskers'], color=color)
#     #plt.setp(bp['caps'], color=color)
#     plt.setp(bp['medians'], color='black')

# ticks = ['X-LIGA', 'DRIE/X-LIGA', 'X-LIGA/DRIE', 'DRIE']

# bpl = plt.boxplot(data_sf, positions=np.array(range(len(data_f_w)))*0.7, sym='', widths=0.15, whis = [1,99], patch_artist = True)
# bpm = plt.boxplot(data_swf, positions=np.array(range(len(data_wf)))*0.7-0.2, sym='', widths=0.15, whis = [1,99], patch_artist = True)
# bpr = plt.boxplot(data_sf_w, positions=np.array(range(len(data_wf)))*0.7+0.2, sym='', widths=0.15, whis = [1,99], patch_artist = True)

# colors = ["#E69F00", "#56B4E9", "#009E73"]

# set_box_color(bpl, colors[0]) # colors are from http://colorbrewer2.org/
# set_box_color(bpm, colors[1])
# set_box_color(bpr, colors[2])

# # draw temporary red and blue lines and use them to create a legend
# plt.plot([], c=colors[1], label='70kVp W spectrum')
# plt.plot([], c=colors[0], label='With 3mm Al filter')
# plt.plot([], c=colors[2], label='With filter and 10cm water')

# plt.xticks(np.array(range(len(data_wf)))*0.7, ticks)
# plt.xlim(-0.35, 2.45)
# #plt.tight_layout()

# plt.ylabel('Sensitivity (a. u.)', fontsize = 10)
# plt.xticks(fontsize=8)
# plt.yticks(fontsize=8)



# plt.axvline(0.35,  color='gray', linewidth=0.5)
# plt.axvline(0.35 + 0.7,  color='gray', linewidth=0.5)
# plt.axvline(0.35 + 2*0.7,  color='gray', linewidth=0.5)

# ax = plt.gca()
# #legend_elements =  lines.Line2D([], [], marker='o', color='w', label='Simulations',  markerfacecolor='black', markersize=8)
# # where some data has already been plotted to ax
# #handles, labels = ax.get_legend_handles_labels()
# #handles.append(legend_elements) 
# # plot the legend
# plt.legend(loc='upper right', fontsize = 6)


# ymin, ymax = plt.gca().get_ylim()

# #plt.legend(fontsize=6)

# #plt.grid('off')
# #plt.grid(axis='y')

# plt.title('Sensitivity distribution comparison', fontsize = 10)
# #plt.savefig('Sensitivity_Boxplots.png', bbox_inches = 'tight')

## Transmission Ratios

In [ ]:
liga_mean_abs_uf = np.mean(trans_ref_liga_uf[18:33])
drie_mean_abs_uf = np.mean(trans_ref_drie_uf[18:33])
drie_liga_mean_abs_uf = np.mean(trans_ref_drie_liga_uf[18:33])
liga_drie_mean_abs_uf = np.mean(trans_ref_liga_drie_uf[18:33])


liga_mean_abs_f = np.mean(trans_ref_liga_f[18:33])
drie_mean_abs_f = np.mean(trans_ref_drie_f[18:33])
drie_liga_mean_abs_f = np.mean(trans_ref_drie_liga_f[18:33])
liga_drie_mean_abs_f = np.mean(trans_ref_liga_drie_f[18:33])


liga_mean_abs_wf = np.mean(trans_ref_liga_wf[18:33])
drie_mean_abs_wf = np.mean(trans_ref_drie_wf[18:33])
drie_liga_mean_abs_wf = np.mean(trans_ref_drie_liga_wf[18:33])
liga_drie_mean_abs_wf = np.mean(trans_ref_liga_drie_wf[18:33])


In [ ]:
print('Unfiltered ratio to X-LIGA')
print('DRIE to X-LIGA: ', liga_mean_abs_uf / drie_mean_abs_uf)
print('DRIE/LIGA to X-LIGA: ', liga_mean_abs_uf / drie_liga_mean_abs_uf)
print('LIGA/DRIE to X-LIGA: ', liga_mean_abs_uf / liga_drie_mean_abs_uf)


print('---------------------')
print('3mm Al Filtered ratio to X-LIGA')
print('DRIE to X-LIGA: ', liga_mean_abs_f / drie_mean_abs_f)
print('DRIE/LIGA to X-LIGA: ', liga_mean_abs_f / drie_liga_mean_abs_f)
print('LIGA/DRIE to X-LIGA: ', liga_mean_abs_f / liga_drie_mean_abs_f)


print('---------------------')
print('3mm Al Filtered + water ratio to X-LIGA')
print('DRIE to X-LIGA: ', liga_mean_abs_wf / drie_mean_abs_wf)
print('DRIE/LIGA to X-LIGA: ', liga_mean_abs_wf / drie_liga_mean_abs_wf)
print('LIGA/DRIE to X-LIGA: ', liga_mean_abs_wf / liga_drie_mean_abs_wf)


# Void simulation loading

Data generated by: Missing_Trenches_Simulation.ipynb is required

In [ ]:
sim_paths = 'PATH/TO/MISSING_TRENCHES_DATA/'
dvis_image_mw = np.load('../data/Flats/dvis_profile_mw_f.npy')

In [ ]:
def calculate_dictionary(simulation_dir, pxSize, sample_thickness = None, mu_sample = None):
    all_sims = os.listdir(simulation_dir)#os.listdir(os.path.join(simulation_dir, '2024', '04'))    

    I = []
    VIS = []
    VIS_STD = []
    SNR = []
    energies = []
    sourceSize = []
    shift_grating = []
    transmission_lines = []
    visibility_lines = []
    phase_lines = []
    p1 = []
    
    for idx, sim in tqdm(enumerate(all_sims)):
        sim_path = Path(os.path.join(simulation_dir, sim))
        try:
            config_dict = config.load(Path(sim_path / 'config.yaml'))
            sp = config_dict["sim_params"]
            detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
            pixel_rectangle = np.abs(detector_x) <= pxSize
    
            wavefronts = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[10000, 70000])
    
            sourceSize.append(2*config_dict["multisource"]["x_range"][1])
            try:
                #shift = config_dict["elements"][0]["grid_path"]
                shift_grating.append(int(config_dict["elements"][0]["grid_path"].split('_')[-1][:-4]))

                p1.append(config_dict["elements"][2]["pitch"])
            except:
                shift_grating.append(0.0)
                p1.append(config_dict["elements"][1]["pitch"])
            phase_steps = len(config_dict["elements"][-1]["x_positions"])

    
            # Calculate the wavefront
            summed_wf = np.zeros_like(wavefronts[0][0])
            energies = []
            for k, point in enumerate(wavefronts):
                wf, x_point, eng = point
                energies.append(eng)
                if sample_thickness:
                    summed_wf += wf * np.exp(-mu_sample(eng) * sample_thickness) 
                else:
                    summed_wf += wf
    
            # Convolve the wavefront
            convolved = []
            for i in range(phase_steps):
                convolved.append(np.convolve(summed_wf[i,:], pixel_rectangle, mode = 'same'))
            convolved = np.asarray(convolved)
            
            #int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, summed_wf[:,1:], p2s[-1], phase_steps, plot_curve = False)
            #vis_noC.append(vis)
            #VIS.append(np.mean(vis[68:148]))
    
            
            int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, convolved[:,:], pxSize, phase_steps, plot_curve = False)
    
            transmission_lines.append(trans)
            visibility_lines.append(vis)
            phase_lines.append(phase)
            
            shape_ = trans.shape[0]
            middle_axis = int(shape_/2)
            idx_start = middle_axis - 14#int(middle_axis / 2)
            idx_end = middle_axis + 14#int(middle_axis / 2)

            #print(idx_start - idx_end)
            #idx_start = 5
            #idx_end = 22
            I.append(np.mean(trans[idx_start:idx_end]))
            VIS.append(np.mean(vis[idx_start:idx_end]))
            VIS_STD.append(np.std(vis[idx_start:idx_end]))
            SNR.append(np.sqrt(1e5*I[-1]) * VIS[-1])

        except Exception as e:
            print(e)
            print(f"rejected {sim}")
            pass
    I = np.asarray(I)
    VIS = np.asarray(VIS)
    VIS_STD = np.asarray(VIS_STD)
    SNR = np.asarray(SNR)
    sourceSize = np.asarray(sourceSize)
    shift_grating = np.asarray(shift_grating)

    visibility_lines = np.asarray(visibility_lines)
    transmission_lines = np.asarray(transmission_lines)
    phase_lines = np.asarray(phase_lines)
    
    
    print(idx_start)
    print(idx_end)
    energies = np.asarray(energies)
    p1 = np.asarray(p1)
    return {
        "I": I,
        "VIS": VIS,
        "VIS_STD": VIS_STD,
        "Signal": SNR,
        "sourceSize": sourceSize,
        "shift_grating": shift_grating,
        "visibility": visibility_lines,
        "transmission": transmission_lines,
        "phase": phase_lines,
        "p1": p1,
        #"energies": energies
        }, energies
    

In [ ]:
dict_plots, energies = calculate_dictionary(sim_paths, 75*1e-6)

In [ ]:
source_sizes = np.array([10, 50, 100, 150, 200])*1e-6

idx_20 = np.where((dict_plots['sourceSize'] == 2*source_sizes[0]))[0].tolist()
idx_100 = np.where((dict_plots['sourceSize'] == 2*source_sizes[1]))[0].tolist()
idx_200 = np.where((dict_plots['sourceSize'] == 2*source_sizes[2]))[0].tolist()
idx_300 = np.where((dict_plots['sourceSize'] == 2*source_sizes[3]))[0].tolist()
idx_400 = np.where((dict_plots['sourceSize'] == 2*source_sizes[4]))[0].tolist()


idx_shift_25 = np.where(dict_plots['shift_grating'] == 25)[0].tolist()
idx_shift_50 = np.where(dict_plots['shift_grating'] == 50)[0].tolist()
idx_shift_75 = np.where(dict_plots['shift_grating'] == 75)[0].tolist()
idx_shift_100 = np.where(dict_plots['shift_grating'] == 100)[0].tolist()
idx_shift_250 = np.where(dict_plots['shift_grating'] == 250)[0].tolist()
idx_shift_300 = np.where(dict_plots['shift_grating'] == 300)[0].tolist()
idx_shift_450 = np.where(dict_plots['shift_grating'] == 450)[0].tolist()
idx_shift_500 = np.where(dict_plots['shift_grating'] == 500)[0].tolist()
idx_shift_650 = np.where(dict_plots['shift_grating'] == 650)[0].tolist()
idx_shift_700 = np.where(dict_plots['shift_grating'] == 700)[0].tolist()

idxs_selected_20 = list(set(idx_20) & (set(idx_shift_100) | set(idx_shift_300) | set(idx_shift_500)))
idxs_selected_1 = list(set(idx_shift_100) & (set(idx_20) | set(idx_100) | set(idx_200) | set(idx_300) | set(idx_400)))


x_pixel = np.linspace(-53/2 * 75, 53/2 * 75, 53)

In [ ]:
dict_plots['shift_grating'][idxs_selected_20]/1e2

In [ ]:
idxs_missing_trench = [3, 2, 1]
names_labels = ['Three', 'Two', 'One']

In [ ]:
plt.figure(figsize=(2.5,2))
for j, idx in enumerate(idxs_selected_20):
    plt.plot(x_pixel[12:-12], dict_plots['visibility'][idx,12:-12]*0.91, label = f"{names_labels[j]} unfilled trenches")
plt.plot(x_pixel[12:-12], dvis_image_mw[160,93-16:93+13], 'o-', label='Measurement')
plt.title('Visibility profile on a detector row', fontsize = 5)
plt.ylabel('Visibility', fontsize = 5)
plt.xlabel('Distance on detector [µm]', fontsize = 5)

plt.grid()
plt.legend(fontsize = 4.5)


In [ ]:
plt.figure(figsize=(2.5,2))

sorted_idxs_1 = np.argsort(dict_plots['sourceSize'][idxs_selected_1])
idxs_selected_1 = [idxs_selected_1[idx] for idx in sorted_idxs_1]

for idx in idxs_selected_1:
    plt.plot(x_pixel[12:-12], dict_plots['visibility'][idx,12:-12]*0.91, label =  f"{dict_plots['sourceSize'][idx]*1e6:.0f} µm")
plt.title('Visibility profile on a detector row for 20 µm source size', fontsize = 5)
plt.ylabel('Visibility', fontsize = 5)
plt.xlabel('Distance on detector [µm]', fontsize = 5)

plt.grid()
plt.legend(fontsize = 4.5)


# Crack Simulation Loading and Analysis

Data generated by: Simulation_Crack.ipynb is required

In [ ]:
simulations_dir = Path('/PATH/TO/SIMULATION_CRACK_DATA/')

In [ ]:
def calculate_dictionary(simulation_dir, pxSize, sample_thickness = None, mu_sample = None):
    #all_sims = os.listdir(os.path.join(simulation_dir, '2023', '12'))    
    all_sims = os.listdir(simulation_dir)
    
    I = []
    VIS = []
    SNR = []
    energies = []
    sourceSize = []
    shift_grating = []
    transmission_lines = []
    visibility_lines = []
    phase_lines = []
    p1 = []
    
    for idx, sim in tqdm(enumerate(all_sims)):
        sim_path = Path(os.path.join(simulation_dir, sim))
        try:
            config_dict = config.load(Path(sim_path / 'config.yaml'))
            sp = config_dict["sim_params"]
            detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
            pixel_rectangle = np.abs(detector_x) <= pxSize
    
            wavefronts = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[10000, 70000])
    
            sourceSize.append(2*config_dict["multisource"]["x_range"][1])
            try:
                shift_grating.append(float(config_dict["elements"][0]["grid_path"][-7:-4]))

                p1.append(config_dict["elements"][2]["pitch"])
            except:
                shift_grating.append(0.0)
                p1.append(config_dict["elements"][1]["pitch"])
            phase_steps = len(config_dict["elements"][-1]["x_positions"])

    
            # Calculate the wavefront
            summed_wf = np.zeros_like(wavefronts[0][0])
            energies = []
            for k, point in enumerate(wavefronts):
                wf, x_point, eng = point
                energies.append(eng)
                if sample_thickness:
                    summed_wf += wf * np.exp(-mu_sample(eng) * sample_thickness) 
                else:
                    summed_wf += wf
    
            # Convolve the wavefront
            convolved = []
            for i in range(phase_steps):
                convolved.append(np.convolve(summed_wf[i,:], pixel_rectangle, mode = 'same'))
            convolved = np.asarray(convolved)
            
            #int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, summed_wf[:,1:], p2s[-1], phase_steps, plot_curve = False)
            #vis_noC.append(vis)
            #VIS.append(np.mean(vis[68:148]))
    
            
            int_px, trans, phase, vis, pxEdges = perform_binned_signal_retrieval(detector_x, convolved[:,:], pxSize, phase_steps, plot_curve = False)
    
            transmission_lines.append(trans)
            visibility_lines.append(vis)
            phase_lines.append(phase)
            
            shape_ = trans.shape[0]
            middle_axis = int(shape_/2)
            idx_start = middle_axis - int(middle_axis / 2)
            idx_end = middle_axis + int(middle_axis / 2)

            print(idx_start - idx_end)
            #idx_start = 5
            #idx_end = 22
            I.append(np.mean(trans[idx_start:idx_end]))
            VIS.append(np.mean(vis[idx_start:idx_end]))
            SNR.append(np.sqrt(1e5*I[-1]) * VIS[-1])

        except Exception as e:
            print(e)
            print(f"rejected {sim}")
            pass
    I = np.asarray(I)
    VIS = np.asarray(VIS)
    SNR = np.asarray(SNR)
    sourceSize = np.asarray(sourceSize)
    shift_grating = np.asarray(shift_grating)

    visibility_lines = np.asarray(visibility_lines)
    transmission_lines = np.asarray(transmission_lines)
    phase_lines = np.asarray(phase_lines)
    
    
    print(idx_start)
    print(idx_end)
    energies = np.asarray(energies)
    p1 = np.asarray(p1)
    return {
        "I": I,
        "VIS": VIS,
        "Signal": SNR,
        "sourceSize": sourceSize,
        "shift_grating": shift_grating,
        "visibility": visibility_lines,
        "transmission": transmission_lines,
        "phase": phase_lines,
        "p1": p1,
        #"energies": energies
        }, energies
    

In [ ]:
dict_plots, energies = calculate_dictionary(simulations_dir, 75*1e-6)
# weight for bridges
dict_plots['VIS'] = dict_plots['VIS'] * 0.91
dict_plots['visibility'] = dict_plots['visibility'] * 0.91
idxs_sort = np.argsort(dict_plots['sourceSize'])

In [ ]:
dict_plots['VIS'].shape
plt.figure(figsize=(3.5,3))
vmin = np.min(dict_plots['VIS'])
vmax = np.max(dict_plots['VIS'])
level_boundaries = np.linspace(vmin, vmax, 5)

scale1 = 1
scale2 = 1e6

sh_grat = np.arange(0, 0.6, 0.012)
s_size = np.arange(20, 320, 6)

c1, c2 = np.meshgrid(sh_grat, s_size)


Ti = griddata((dict_plots['shift_grating']*scale1, dict_plots['sourceSize']*scale2), dict_plots['VIS'], (c1, c2), method='linear')

im = plt.contourf(c1, c2, Ti, levels = 20, vmin = vmin, vmax = vmax, cmap = 'inferno')
cbar = plt.colorbar(im,  ticks=level_boundaries)

cbar.ax.tick_params(axis='both', which='both', labelsize=8)
cbar.set_ticks([0.07, 0.12, 0.17])
plt.xlim(dict_plots['shift_grating'].min()*scale1, dict_plots['shift_grating'].max()*0.985*scale1)
plt.ylim(dict_plots['sourceSize'].min()*scale2, dict_plots['sourceSize'].max()*0.985*scale2)

plt.tick_params(axis='x', labelsize=8)
plt.tick_params(axis='y', labelsize=8)
plt.gca().set_aspect('auto')
plt.title('Visibility', fontsize = 8)
plt.xlabel(f'Shift as a fraction of 2π', fontsize = 8)
plt.ylabel(f'Source size [µm]', fontsize = 8)
#plt.savefig('Visibility_for_different_shifts.png', bbox_inches = 'tight')